# TO-DO: 

- Explore 4 regression models, 1 as baseline and the others more advanced
- Train test splits
- Apply cross validation and hyperparamter tunning in at least 2 models
- Get the feature importance and see which feature is more relevant
- Apply cross validation for evaluation
- Draw a table comparing the performances of each model. Example:

# Model Comparison

In this section, we compare the performance of different models using various metrics. The models evaluated include [Model 1], [Model 2], and [Model 3]. The following metrics are used for comparison:

- **Accuracy**: The ratio of correctly predicted observations to the total observations. It is a useful metric when the classes are well balanced.
- **Precision**: The ratio of correctly predicted positive observations to the total predicted positives. High precision relates to the low false positive rate.
- **Recall (Sensitivity)**: The ratio of correctly predicted positive observations to all observations in the actual class. High recall relates to the low false negative rate.
- **F1 Score**: The weighted average of Precision and Recall. It is a better metric than accuracy for imbalanced datasets.
- **ROC-AUC Score**: Area Under the Receiver Operating Characteristic Curve, which is a performance measurement for classification problems at various threshold settings.

## Model Metrics

| **Model**       | **Accuracy** | **Precision** | **Recall** | **F1 Score** | **ROC-AUC Score** |
|-----------------|--------------|---------------|------------|--------------|-------------------|
| **Model 1**     | 0.XX         | 0.XX          | 0.XX       | 0.XX         | 0.XX              |
| **Model 2**     | 0.XX         | 0.XX          | 0.XX       | 0.XX         | 0.XX              |
| **Model 3**     | 0.XX         | 0.XX          | 0.XX       | 0.XX         | 0.XX              |

## Interpretation of Results

- **Model 1**: [Brief summary of performance]
- **Model 2**: [Brief summary of performance]
- **Model 3**: [Brief summary of performance]

Based on the comparison, we can observe that [Model X] performs the best overall due to its higher [Metric(s)]. However, depending on the specific use case and requirements (e.g., higher precision vs. recall), [Model Y] might be preferable.

## Conclusion

The chosen model for deployment is [Model Z] due to its superior performance in [specific metric(s)]. Further fine-tuning and validation will be conducted to ensure robustness and reliability in a real-world scenario.


**OUTPUT**: The best performer model saved in a pickle file in '..\..\repos\ai-logistics\models\price_predictor'

In [28]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score


In [29]:
X_pca=pd.read_csv('../../data/price_predictor/processed/X_pca.csv')
y=pd.read_csv('../../data/price_predictor/processed/y.csv')

**Baseline Model**

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

In [40]:
lr_model = LinearRegression()

lr_model.fit(X_train, y_train)

y_pred_lr = lr_model.predict(X_test)

lr_mse = mean_squared_error(y_test, y_pred_lr)
lr_rmse = np.sqrt(lr_mse)
lr_r2 = r2_score(y_test, y_pred_lr)

print(f'Linear Regression RMSE: {lr_rmse}')
print(f'Linear Regression R2: {lr_r2}')

Linear Regression RMSE: 30.034397689991362
Linear Regression R2: -0.030678180531177768


In [41]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(),
    'SVR': SVR(),
    'K-Nearest Neighbors': KNeighborsRegressor(),
    'Gradient Boosting': GradientBoostingRegressor()
}

results = {}
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)  # Calculate RMSE
    r2 = r2_score(y_test, y_pred)
    results[model_name] = {
        'Mean Squared Error': mse,
        'Root Mean Squared Error': rmse,
        'R^2 Score': r2
    }

for model_name, metrics in results.items():
    print(f"Model: {model_name}")
    print(f"Mean Squared Error: {metrics['Mean Squared Error']}")
    print(f"Root Mean Squared Error: {metrics['Root Mean Squared Error']}")
    print(f"R^2 Score: {metrics['R^2 Score']}\n")


/Users/daniel/anaconda3/envs/DataScienceFoundations/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/daniel/anaconda3/envs/DataScienceFoundations/lib/python3.11/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/daniel/anaconda3/envs/DataScienceFoundations/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:668: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)  # TODO: Is this still required?


Model: Linear Regression
Mean Squared Error: 902.0650446005585
Root Mean Squared Error: 30.034397689991362
R^2 Score: -0.030678180531177768

Model: Random Forest
Mean Squared Error: 909.2088544156146
Root Mean Squared Error: 30.15309029627999
R^2 Score: -0.038840528630480486

Model: SVR
Mean Squared Error: 877.3263591929172
Root Mean Squared Error: 29.619695460840195
R^2 Score: -0.002412343807650208

Model: K-Nearest Neighbors
Mean Squared Error: 1001.7053044245808
Root Mean Squared Error: 31.64972834677386
R^2 Score: -0.14452478429637772

Model: Gradient Boosting
Mean Squared Error: 906.2182895827636
Root Mean Squared Error: 30.103459761010257
R^2 Score: -0.03542357999785928

